# Introduction

<br>

## Data set
- A dataset of used cars from eBay Kleinanzeigen(a online classifieds section of the German eBay website) on Kaggle
    - Originally Scraped in Kaggle
    - Made few modification in this analysis
        - sampled 50,000 data randomly
        - dirtied to be a sample of data clean practice(original ver on kaggle has been cleaned already)  

<br>

## Aim of this analysis
- To clean the data and analyze the included used car listings
- To become familiar with data clean techniques with Pandas
<br>

---
# Preperation
## importing libraries(Pandas, Numpy)
## reading file(autos.csv)



In [1]:
import pandas as pd
import numpy as np

autos = pd.read_csv("autos.csv", encoding="UTF-8")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xdc in position 23: invalid continuation byte

Got an UnicodeDecodeError(encoding error) in reading autos.csv.  

**possible cause**  
    encoded by encodings except UTF-8

<br>

- try next encodings

In [ ]:
autos = pd.read_csv("autos.csv", encoding="Latin-1")

successed to reading autos.csv.

<br>

## confim the data frame

<br>

### autos table(dictionary definition)  

| key | type | description |
| --- | --- | --- |
| `dataCrawled` | string | When this ad was first crawled(All values taken from this date) |
| `name` | string | Name of the car |
| `seller` | string | Whether the seller is private or a dealer |
| `offerType` | string | The type of listing |
| `price` | string | The price on the ad to sell the car |
| `abtest` | string | Whether the listing is included in an A/B test |
| `vehicleType` | string | The vehicle Type |
| `yearOfRegistration` | int | The year in which the car was first registered |
| `gearbox` | string | The transmission type |
| `powerPS` | int | The power of the car in PS( Pferde Stärke: 馬力) |
| `model` | string | The car model name |
| `kilometer` | string | How long the car has driven |
| `monthOfRegistration` | int | The month in which the car was first registered |
| `fuelType` | string | What type of fuel the car uses |
| `brand` | string | The brand of the car |
| `notRepairedDamage` | string | If the car has a damage which is not yet repaired |
| `dateCreated` | string | The date on which the eBay listing was created |
| `nrOfPictures` | int | The number of pictures in the ad |
| `postalCode` | int | The postal code for the location of the vehicle |
| `lastSeenOnline` | string | When the crawler saw this ad last online |


In [ ]:
autos # itself

In [ ]:
autos.info() # print dataframe

In [ ]:
autos.head() # first 5 rows

### Brief data set observation
#### overview
- 20 columns(15 strings, 5 ints)
- less than 20% null values in each columns if exists
- camelcase column names, not snakecase

#### As is 
- camelcase

- missing values
    - vehicleType    44905 non-null
    - gearbox     47320 non-null
    - model     47242 non-null
    - fuelType     45518 non-null
    - notRepairedDamage     40171 non-null

- unpreferable data input to analize
    - name
        - too much long to read & has multiple info
    - price
        - "$0000"
    - power PS
        - some may have 0 values
    - odmeter
        - "0000km"
    - notRepairedDamage
        - "Nein"

<br>

#### To be
- convert
    - camelcase to snakecase
- split?
    - name
- astype
    - price
    - odmeter
- complement
    - power PS
- replace
    - notRegisteredDamage
- missing values complement / drop
    - vehcleType
    - gearbox
    - model
    - fuelType
    - notRegisteredDamage

<br>

---
# Cleaning Data
## Column names
- camelcase to snakecase
- reword names to be more descriptive

In [ ]:
# print an array of existing columns
print(autos.columns)

In [ ]:
# copy the array and rename those names
autos.columns = ['date_crawled', 'name', 'seller', 'offer_type', 'price', 'ab_test',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'num_photos', 'postal_code',
       'last_seen']
autos.head()

## Initial Exploration and Cleaning
### Consideration
- what other cleaning tasks needed
    - drop text columns with most values are the same

### Exploration


In [ ]:
# look at statistics for all columns
autos.describe(include="all")

#### overview
- columns most values has same value
    - seller
    - offer_type
    
- numeric data stored as text
    - price
    - odometer

- columns that need more investigation
    - power_ps
        - 0 PS car exists?
    - num_photos
        - 0 statistics?

#### Investigation
- `power_ps`

- `num_photos`

In [ ]:
autos["power_ps"].value_counts()

In [ ]:
autos["num_photos"].value_counts()

- `power_ps`  
    should replace??  
    1 for many rows
    
- `num_photos`  
    should drop this cloumn  
    0 for every column

## drop meanless cloumns

In [ ]:
autos = autos.drop(["seller", "offer_type", "num_photos"], axis=1)

## clean numeric data stored as text
- target : `price`, `odometer`
- to do : 
    - remove non-numeric characters
    - convert the column to a numeric dtype
    - rename the `odometer` to `odometer_km`

In [ ]:
# price
autos["price"] = (autos["price"]
                  .str.replace("$", "")
                  .str.replace(",", "")
                  .astype(int)
                    )


In [ ]:
autos["price"].head()

In [ ]:
# odometer
autos["odometer"] = (autos["odometer"]
                  .str.replace("km", "")
                  .str.replace(",", "")
                  .astype(int)
                    )
autos.rename({"odometer":"odometer_km"}, axis=1, inplace=True)

In [ ]:
autos["odometer_km"].head()

# Exploration

In [ ]:
print("***unique values*** \n")
print(autos["price"].unique().shape)
print("\n")
print("***statics*** \n")
print(autos["price"].describe())
print("\n")
print("***counted values*** \n")
autos["price"].value_counts().head(20)

- `price`
    - meanful info
        - alomost all values are nice rounded numbers
        - 2357 various range of pricing, on the other hand
            - people's decision are reflected?? not decided by site??
    - meanless info
        - 1421 $0 cars in 50000 cars
            - should be drop the rows??

In [ ]:
autos["price"].value_counts().sort_index(ascending=False).head(30)

In [ ]:
autos["price"].value_counts().sort_index(ascending=True).head(30)

Given that eBay is an auction site, there could legitimately be items where the opening bid is 1.  
  
Many people will keep the 1 dalar items, but remove anything above 350,000, since it seems that prices increase steadily to that number and then jump up to less realistic numbers.

In [ ]:
autos = autos[autos["price"].between(1,351000)]
autos["price"].describe()

In [ ]:
print("***unique values*** \n")
print(autos["odometer_km"].unique().shape)
print("\n")
print("***statics*** \n")
print(autos["odometer_km"].describe())
print("\n")
print("***counted values*** \n")
autos["odometer_km"].value_counts().head(20)

- `odometer_km`
    - meanful info
        - more than a half of cars has driven 150000km
        - nice rounded numbers to be seen
            - might be just rounded actually
            - means this column has anbiguous info  
            
## columns with date dtype
- `date_crawled`
- `registration_month`
- `registration_year`
- `ad_created`
- `last_seen`  
    These are a combination of dates that were crawled, and with meta-information from the crawler.  
    The non-registration dates are stored as strings.



In [ ]:
autos[['date_crawled','ad_created','last_seen']][0:5]

In [ ]:
(autos["date_crawled"]
        .str[:10]
        .value_counts(normalize=True, dropna=False)
        .sort_index()
        )

In [ ]:
(autos["date_crawled"]
        .str[:10]
        .value_counts(normalize=True, dropna=False)
        .sort_values()
        )

In [ ]:
(autos["last_seen"]
        .str[:10]
        .value_counts(normalize=True, dropna=False)
        .sort_index()
        )

In [ ]:
print(autos["ad_created"].str[:10].unique().shape)
(autos["ad_created"]
        .str[:10]
        .value_counts(normalize=True, dropna=False)
        .sort_index()
        )

Looks like the site was crawled daily over roughly a one month period in March and April 2016.   
The distribution of listings crawled on each day is roughly uniform.  
The crawler recorded the date it last saw any listing, which allows us to determine on what day a listing was removed, presumably because the car was sold.

The last three days contain a disproportionate amount of 'last seen' values.   Given that these are 6-10x the values from the previous days, it's unlikely that there was a massive spike in sales, and more likely that these values are to do with the crawling period ending and don't indicate car sales.  

There is a large variety of ad created dates. Most fall within 1-2 months of the listing date, but a few are quite old, with the oldest at around 9 months.

In [ ]:
autos["registration_year"].describe()

The year that the car was first registered will likely indicate the age of the car. Looking at this column, we note some odd values.   
The minimum value is 1000, long before cars were invented and the maximum is 9999, many years into the future.  

### Incorrect Registration Year Data
ecause a car can't be first registered before the listing was seen, any vehicle with a registration year above 2016 is definitely inaccurate. Determining the earliest valid year is more difficult. Realistically, it could be somewhere in the first few decades of the 1900s.

One option is to remove the listings with these values. Let's determine what percentage of our data has invalid values in this column:

In [2]:
(~autos["registration_year"].between(1900,2016)).sum() / autos.shape[0]

NameError: name 'autos' is not defined

Given that this is less than 4% of our data, we will remove these rows.

In [ ]:
# Many ways to select rows in a dataframe that fall within a value range for a column.
# Using `Series.between()` is one way.
autos = autos[autos["registration_year"].between(1900,2016)]
autos["registration_year"].value_counts(normalize=True).head(10)

It appears that most of the vehicles were first registered in the past 20 years.

### Exploring Price by Brand

In [ ]:
autos["brand"].value_counts(normalize=True)

German manufacturers represent four out of the top five brands, almost 50% of the overall listings. Volkswagen is by far the most popular brand, with approximately double the cars for sale of the next two brands combined.

There are lots of brands that don't have a significant percentage of listings, so we will limit our analysis to brands representing more than 5% of total listings.

In [3]:
brand_counts = autos["brand"].value_counts(normalize=True)
common_brands = brand_counts[brand_counts > .05].index
print(common_brands)

NameError: name 'autos' is not defined

In [ ]:
brand_mean_prices = {}

for brand in common_brands:
    brand_only = autos[autos["brand"] == brand]
    mean_price = brand_only["price"].mean()
    brand_mean_prices[brand] = int(mean_price)

brand_mean_prices

Of the top 5 brands, there is a distinct price gap:

- Audi, BMW and Mercedes Benz are more expensive
- Ford and Opel are less expensive
- Volkswagen is in between - this may explain its popularity, it may be a 'best of 'both worlds' option.  

### Exploring Mileage

In [ ]:
bmp_series = pd.Series(brand_mean_prices)
pd.DataFrame(bmp_series, columns=["mean_mileage"])

In [4]:
brand_mean_mileage = {}

for brand in common_brands:
    brand_only = autos[autos["brand"] == brand]
    mean_mileage = brand_only["odometer_km"].mean()
    brand_mean_mileage[brand] = int(mean_mileage)

mean_mileage = pd.Series(brand_mean_mileage).sort_values(ascending=False)
mean_prices = pd.Series(brand_mean_prices).sort_values(ascending=False)

NameError: name 'common_brands' is not defined

In [ ]:
brand_info = pd.DataFrame(mean_mileage,columns=['mean_mileage'])
brand_info

In [ ]:
brand_info["mean_price"] = mean_prices
brand_info

The range of car mileages does not vary as much as the prices do by brand,   instead all falling within 10% for the top brands.  
There is a slight trend to the more expensive vehicles having higher mileage, with the less expensive vehicles having lower mileage.